In [1]:
#————————————————————

# Name: OpenAI Fine-Tuning(V1)
# Purpose: Create a fine-tuned openai GPT model that assist in the classification of products.
# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 10.01.2024
# Last Updated: 10.01.2024
# Python Version: 3.10.4

#General Sources:
#https://platform.openai.com/docs/api-reference?lang=python

#Openai Usage:
#https://platform.openai.com/usage

#Additionals:

#https://world.openfoodfacts.org/data

# Download Python packages (run the below command in terminal if packages have not yet been installed)
#pip install -r C:\Python\openai-lab\support\requirements\requirements.txt

#————————————————————

hello


In [1]:
# Import required libraries
import pandas as pd


In [11]:
# Create assistant description (prompt / instructions)

path_input = r"C:\Python\data\en.openfoodfacts.org.products.csv" #Change path if required
df = pd.read_csv(path_input , sep='\t', on_bad_lines='skip', low_memory=False)

In [12]:
df_products = df[["product_name", "categories_tags", "countries_en"]]
df_products = df_products[df_products.countries_en.isin(["United States"])]
df_products = df_products[["product_name", "categories_tags"]]
df_products = df_products.dropna(subset=['product_name'])
df_products = df_products.drop_duplicates(subset ='product_name')
df_products = df_products.dropna(subset=['categories_tags'])
df_products = df_products.loc[df_products['categories_tags'].str.match("^[^,\n]*((,[^,\n]*){3}$)")] #only show rows with category tags with 3 levels, i.e. must have 3 commas.
df_products = df_products.reset_index(drop=True)

In [4]:
categories_tags = df_products["categories_tags"].str.split(",", n = 5, expand = True)
categories_tags = categories_tags[[0,1,2]]
categories_tags.rename(columns = {0:'level_1', 1:'level_2', 2:'level_3'}, inplace = True)

categories_tags["level_1"] = categories_tags["level_1"].str.replace(r'^en:', '', regex=True)
categories_tags["level_2"] = categories_tags["level_2"].str.replace(r'^en:', '', regex=True)
categories_tags["level_3"] = categories_tags["level_3"].str.replace(r'^en:', '', regex=True)

In [5]:
df_products = df_products.join(categories_tags)
df_products = df_products.drop(["categories_tags"], axis = 1)

In [12]:
# Reduce size due to openai limitations

df_products = df[df.countries_en.isin(["United States"])]
df_products = df_products.iloc[:100000]
# df_products = df_products.dropna(subset=['product_name'])
# df_products = df_products.drop_duplicates(subset ='product_name')

In [13]:
#path_output = r"C:\Python\openai-lab\data\openfoodfacts.csv"
path_output = r"C:\Python\data\openfoodfacts.csv" #Change path if required

#df_products.to_excel(path_output, sheet_name='Sheet1', index = False)
df_products.to_csv(path_output, sep='\t', encoding='utf-8', index=False)